In [19]:
import os
import shutil
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import nibabel as nib

# Data Organization

In [20]:
from IPython.display import clear_output
def printUpdate(count, total):
  clear_output(wait=True)
  percent = 100.*count/total
  print(f"{count}/{total} ({percent:.2f}%)", end="")
  print(f'[{"██"*int(percent//10)}{"--"*int((100-percent)//10)}]')



for i in range(10):
  printUpdate(i, 10)

9/10 (90.00%)[██████████████████--]


In [21]:
def moveRename(source, dest, num_files = 291):
    """
    Moves all .hdr and .img images from source location to a single directory
    """
    rawDirPat = r"(?:[\W\S]+?)OAS2_([0-9]{4})_MR([0-9]{1})/RAW"
    i = 0
    for root, dir, files in os.walk(source):
        r_match = re.findall(rawDirPat, root)
        if len(r_match) > 0:
            subID = r_match[0][0]
            session = r_match[0][1]
            new_name = f"{subID}_{session}"
            for f in files:
                fname, fext = os.path.splitext(f)
                if fext == ".img" or fext == ".hdr":
                    f_match = re.findall(r"mpr-([0-9]{1}).nifti", fname)
                    if len(f_match) > 0:
                        f_num = f_match[0]
                        old_name = os.path.join(root, f)
                        new_name = os.path.join(
                            dest, f"OAS2_{subID}_MR{f_num}_V{session}.nifti{fext}"
                        )

                        printUpdate(i, num_files)
                        i += 1
                        shutil.copy2(old_name, new_name)
    print()

In [22]:
def convertToNii(source, num_files = 291):
    """
    Convert images from .img to .nii format and get rid of .img and .hdr files.
    """

    i = 0
    for root, dir, files in os.walk(source):
        for f in files:
            fbase, fext = os.path.splitext(f)
            if fext == ".img":
                # print(f"Converting {f}")
                printUpdate(i, num_files)
                i += 1
                fname = os.path.join(root, f)
                img = nib.load(fname)
                nib.save(img, fname.replace(".img", ".nii"))
                os.remove(os.path.join(root, fbase + ".hdr"))
                os.remove(os.path.join(root, fbase + ".img"))
    print()

In [23]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import misc

def toRGB(data):
    x, y = data.shape[:2]
    data = (data-data.min())/(data.max()-data.min())

    img_arr = np.empty(shape=(x,y,4))
    img_arr[:, :, :3] = data
    img_arr[:, :, 3] = 1.
    return img_arr

def convertToJPG(source, num_files = 291):
    """
    Convert images from .nii to .jpg format
    """

    i = 0
    shape = (0,0,0,0)
    for root, dir, files in os.walk(source):
        for f in files:
            fbase, fext = os.path.splitext(f)
            if fext == ".nii":
                # print(f"Converting {f}")
                printUpdate(i, num_files)
                i += 1
                fname = os.path.join(root, f)
                img = nib.load(fname)
                data = img.get_fdata()[116,:,:]

                img_arr = toRGB(data)
                shape = img_arr.shape
                plt.imsave(fname.replace(".nifti.nii", ".jpg"), img_arr)
    return shape


In [24]:
def removeVisits3to5(source):
    """
    Removes all images taken at visits 3-5
    """
    for root, dir, files in os.walk(source):
        for f in files:
            m = re.match(r"OAS2_[0-9]{4}_MR[0-9]{1}_V([0-9]{1})", f)
            session_num = int(m.groups()[0])
            if session_num > 2:
                print(f"Removing {f}")
                os.remove(os.path.join(root, f))

def removeSubjects(subjectIDs:list, source):
    for root, dir, files in os.walk(source):
        for f in files:
            for id in subjectIDs:
                m = re.match(re.compile("("+id+")"), f)
                if m!= None:
                    subjectIDs.remove(id)
                    os.remove(os.path.join(root, f))


In [25]:
# Only run once, to move data
# Set True to move files
move_files = False

In [26]:
if move_files:
  moveRename("datasets/OAS2", "datasets/OAS2_nii", 2733)

In [27]:
if move_files:
  convertToNii("datasets/OAS2_nii", 1366)

In [28]:
if move_files:
  data_shape = convertToJPG("datasets/OAS2_nii", 1366)


In [29]:
if move_files:
  removeVisits3to5("datasets/OAS2_nii")


In [30]:
# Remove subject MRIs who had an age > 95
if move_files:
  removeSubjects(["OAS2_0051_MR3", "OAS2_0087_MR1", "OAS2_0087_MR2"], os.getenv("OAS2NII"))

# Load Data

In [31]:
def moveByClass(df, num_files = 291):
  train_path = os.path.join(os.getenv("OAS2NII"), "train")
  test_path = os.path.join(os.getenv("OAS2NII"), "test")
  dem_path = os.path.join(os.getenv("OAS2NII"), "class_demented")
  ndem_path = os.path.join(os.getenv("OAS2NII"), "class_nondemented")
  if not os.path.exists(train_path):
    os.makedirs(train_path)
    if not os.path.exists(os.path.join(train_path, "class_demented")):
      os.makedirs(os.path.join(train_path, "class_demented"))
    if not os.path.exists(os.path.join(train_path, "class_nondemented")):
      os.makedirs(os.path.join(train_path, "class_nondemented"))
  if not os.path.exists(test_path):
    os.makedirs(test_path)
    if not os.path.exists(os.path.join(test_path, "class_demented")):
      os.makedirs(os.path.join(test_path, "class_demented"))
    if not os.path.exists(os.path.join(test_path, "class_nondemented")):
      os.makedirs(os.path.join(test_path, "class_nondemented"))

  new_paths = []
  i = 0
  for index, row in df.iterrows():
    old_name = row["file"]
    fname = os.path.split(old_name)[1]
    if row["Group"] == 0:
      new_name = os.path.join(os.getenv("OAS2NII"), row["Split"], "class_nondemented", fname)
    else:
      new_name = os.path.join(os.getenv("OAS2NII"), row["Split"], "class_demented", fname)
    new_paths += [new_name]
    printUpdate(i, num_files)
    i += 1
    os.rename(old_name, new_name)
    print(f'Old: {old_name}\nNew:{new_name}\n')

  files = [f for f in os.listdir(os.getenv("OAS2NII")) if os.path.isfile(os.path.join(os.getenv("OAS2NII"), f))]
  # for f in files:
  #   os.remove(os.path.join(os.getenv("OAS2NII"),f))

  df["file"] = new_paths


In [32]:
load_dotenv()
df = pd.read_excel("OAS2-normalized.xlsx")

In [33]:
fnames = []

df.set_index("MRI ID", inplace=True)

for index, row in df.iterrows():
  fname = index + "_V" + str(row["Visit"]+1) +  ".jpg"
  fnames += [os.path.join(os.getenv("OAS2NII"),fname)]

df["file"] = fnames


In [34]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=["Group"]), df["Group"], test_size=0.2, stratify=df[['Group',"Sex_F"]])



# x_train["Split"] = ["Train"]*x_train.shape[0]
# # y_train["Split"] = ["Train"]*y_train.shape[0]
# x_test["Split"] = ["Test"]*x_test.shape[0]
# y_test["Split"] = ["Test"]*y_test.shape[0]


In [35]:
train = x_train.copy(deep=True)
train["Split"] = ["train"]*train.shape[0]
train["Group"] = y_train.values

test = x_test.copy(deep=True)
test["Split"] = ["test"]*test.shape[0]
test["Group"] = y_test.values

df = pd.merge(train, test, how="outer")


In [ ]:
# moveByClass(df)


0/291 (0.00%)[--------------------]


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/OAS2_nii/OAS2_0001_MR1_V1.jpg' -> 'datasets/OAS2_nii/train/class_nondemented/OAS2_0001_MR1_V1.jpg'

# Keras

In [ ]:
# Needed to add cuda to path for GPU utilization
# !source ~/.profile

In [ ]:
import keras
from keras import layers
from keras import ops
import SimpleITK as sitk

In [ ]:

dset_train = keras.utils.image_dataset_from_directory(directory=os.path.join(os.getenv("OAS2NII"),"train"), labels="inferred")
dset_test = keras.utils.image_dataset_from_directory(directory=os.path.join(os.getenv("OAS2NII"),"test"), labels="inferred")


In [ ]:
img_inputs = keras.Input(shape=(256,256,3,))

x = layers.Dense(64, activation="relu")(img_inputs)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
model = keras.Model(inputs=img_inputs, outputs=outputs, name="oas2_model")


In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.categorical_hinge(),
    # List of metrics to monitor
    metrics=[keras.metrics.AUC()],
)

print("Fit model on training data")
history = model.fit(
    dset_train,
    batch_size=64,
    epochs=5,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    # validation_data=(x_val, y_val),
)